In [1]:
!pip install transformers datasets evaluate sacrebleu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 93.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 49.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 107.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 29.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 79.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 32.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 39.5 MB/s eta 0:00:00
     ━━━━━━━━━━

In [2]:
from huggingface_hub import notebook_login

notebook_login()

In [3]:
from datasets import load_dataset

translation = load_dataset("Hansollll/Translation")

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/8000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/Hansollll___parquet/Hansollll--Translation-b286a9fc69f1fc39/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [27]:
translation['train']['translation'][:5]

[{'en': 'In reality, there are many cases in which property ownership tax is operated as a national tax in some countries.',
  'ko': '현실적으로도 나라에 따라서는 재산보유세가 국세로 운영되고 있는 사례가 다수 발견된다.'},
 {'en': 'In order to understand the merits of the summary judgment procedure as a police-led minor case handling procedure, the US and UK minor case handling procedures, which have been models of the summary judgment procedure in Korea, are reviewed below.',
  'ko': '경찰 주도형 경미 사건 처리 절차로서 즉결심판절차의 장점을 이해하기 위해, 아래에서는 우리나라 즉결심판절차의 모델이 되었던 미국과 영국의 경미 사건 처리 절차를 살펴본다.'},
 {'en': 'At the time of the campaign, it was found that there was a difference in the career management area, which is a sub-area of job competency.',
  'ko': '전역 당시 계급에는 직무 역량의 하위 영역인 진로 관리 영역에서 차이를 보이는 것으로 나타났다.'},
 {'en': "As Korean capitalism develops in the direction of extreme individualism, the gap between economic growth and people's quality of life is growing.",
  'ko': '이처럼 한국 자본주의가 극단적인 개인주의 방향으로 발전함에 따라 경제성장과 국민생활의 괴리가 커지고 있다.'},
 {

In [6]:
from transformers import MarianMTModel, MarianTokenizer

src_text = [
    ">>hrv<< 너는 정말로 큰 실수를 저질렀어.",
    ">>hbs<< 나는 지금까지 너무 바빳어."
]

model_name = "Helsinki-NLP/opus-mt-ko-en"
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:194: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [7]:
source_lang = "ko"
target_lang = "en"
prefix = "translate English to French: "

def preprocess_function(examples):
    inputs = [prefix + example[source_lang] for example in examples["translation"]]
    targets = [example[target_lang] for example in examples["translation"]]
    model_inputs = tokenizer(inputs, text_target=targets, max_length=128, truncation=True)
    return model_inputs

In [8]:
tokenized_translation = translation.map(preprocess_function, batched=True)

Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [9]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model_name)

In [10]:
import evaluate

metric = evaluate.load("sacrebleu")

In [11]:
import numpy as np


def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels

In [12]:
def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

In [13]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer

model = MarianMTModel.from_pretrained(model_name)

In [14]:
training_args = Seq2SeqTrainingArguments(
    output_dir="translation_model",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=10,
    predict_with_generate=True,
    # fp16=True,
    eval_steps=500,
    save_steps=500,
    push_to_hub=True
)

In [17]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_translation["train"],
    eval_dataset=tokenized_translation["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

/content/translation_model is already a clone of https://huggingface.co/Hansollll/translation_model. Make sure you pull the latest changes with `repo.git_pull()`.


In [18]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Bleu,Gen Len
1,1.700400,1.506863,30.710800,28.816000
2,1.300200,1.469511,31.448400,28.474000
3,1.054000,1.474053,31.769100,28.524000
4,0.895100,1.482427,31.745600,28.782000
5,0.745000,1.501980,32.325700,28.748500
6,0.628900,1.526219,32.381700,28.715000
7,0.561900,1.541863,32.506700,28.536000
8,0.523200,1.556792,32.488200,28.620500
9,0.469200,1.569964,32.531200,28.682000
10,0.436900,1.572936,32.628200,28.702000


Adding files tracked by Git LFS: ['source.spm', 'target.spm']. This may take a bit of time if the files are large.


TrainOutput(global_step=20000, training_loss=0.8361831039428711, metrics={'train_runtime': 4583.0629, 'train_samples_per_second': 17.456, 'train_steps_per_second': 4.364, 'total_flos': 1156069837504512.0, 'train_loss': 0.8361831039428711, 'epoch': 10.0})

In [19]:
text = "한글을 영어로 번역: 콩과는 질소 고정 박테리아와 자원을 공유한다."

In [21]:
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)

MarianMTModel(
  (model): MarianModel(
    (shared): Embedding(65001, 512, padding_idx=65000)
    (encoder): MarianEncoder(
      (embed_tokens): Embedding(65001, 512, padding_idx=65000)
      (embed_positions): MarianSinusoidalPositionalEmbedding(512, 512)
      (layers): ModuleList(
        (0-5): 6 x MarianEncoderLayer(
          (self_attn): MarianAttention(
            (k_proj): Linear(in_features=512, out_features=512, bias=True)
            (v_proj): Linear(in_features=512, out_features=512, bias=True)
            (q_proj): Linear(in_features=512, out_features=512, bias=True)
            (out_proj): Linear(in_features=512, out_features=512, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation_fn): SiLUActivation()
          (fc1): Linear(in_features=512, out_features=2048, bias=True)
          (fc2): Linear(in_features=2048, out_features=512, bias=True)
          (final_layer_norm): LayerNorm((512,),

In [22]:
inputs = tokenizer(text, return_tensors="pt").input_ids.to(device)
outputs = model.generate(inputs, max_new_tokens=40, do_sample=True, top_k=30, top_p=0.95)

In [23]:
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

With soybeans, nitrogen-fixed bacteria and resources are shared.


In [24]:
text

'한글을 영어로 번역: 콩과는 질소 고정 박테리아와 자원을 공유한다.'

In [25]:
def text_translation(text):
    inputs = tokenizer('한글을 영어로 번역: '+text, return_tensors="pt").input_ids.to(device)
    outputs = model.generate(inputs, max_new_tokens=40, do_sample=True, top_k=30, top_p=0.95)
    print(tokenizer.decode(outputs[0], skip_special_tokens=True))

In [26]:
text_translation('마지막에는 주요 주제를 회상하면서 조용히 마친다.')

At the end, you will calmly finish by reflecting on the main topics.
